<a href="https://colab.research.google.com/github/KhanilReddy/ML1/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U -q 'google-genai'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 1.8 MB/s eta 0:00:00


In [1]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [49]:
from google.colab import userdata
from google import genai
from google.genai import types
from pydantic import BaseModel
import json
import time

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

system_instructions = """
You are an AI model designed to classify IT support tickets.
Classify each ticket based on the following attributes:
- Department: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
- Priority: low, medium, high
- Type: Incident, Request, Problem, Change
"""

# Define the expected JSON structure for the ticket classification
class TicketClassification(BaseModel):
    ticket_ID: str
    ticket_body: str
    ticket_subject: str
    priority: str
    ticket_type: str

# Initialize the GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Load the testing data set
with open('testing_data_set.json', 'r') as file:
    test_json = json.load(file)


final_dict = {}

# Function to classify a ticket
def classify_ticket(ticket: dict) -> dict:
    prompt = f"{system_instructions}\n\nTicket:\nSubject: {ticket['ticket_subject']}\nBody: {ticket['ticket_body']}"

    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': TicketClassification,
        },
    )
    result = response.parsed
    final_dict[ticket['ticket_ID']] = [None, result.ticket_type, result.priority, None]

# Classify tickets in the specified range
for i in range(0,500):
    print(f"Classifying ticket {i}")
    classify_ticket(test_json[i])
    time.sleep(3)


Classifying ticket 0
Classifying ticket 1
Classifying ticket 2
Classifying ticket 3
Classifying ticket 4
Classifying ticket 5
Classifying ticket 6
Classifying ticket 7
Classifying ticket 8
Classifying ticket 9
Classifying ticket 10
Classifying ticket 11
Classifying ticket 12
Classifying ticket 13
Classifying ticket 14
Classifying ticket 15
Classifying ticket 16
Classifying ticket 17
Classifying ticket 18
Classifying ticket 19
Classifying ticket 20
Classifying ticket 21
Classifying ticket 22
Classifying ticket 23
Classifying ticket 24
Classifying ticket 25
Classifying ticket 26
Classifying ticket 27
Classifying ticket 28
Classifying ticket 29
Classifying ticket 30
Classifying ticket 31
Classifying ticket 32
Classifying ticket 33
Classifying ticket 34
Classifying ticket 35
Classifying ticket 36
Classifying ticket 37
Classifying ticket 38
Classifying ticket 39
Classifying ticket 40
Classifying ticket 41
Classifying ticket 42
Classifying ticket 43
Classifying ticket 44
Classifying ticket 4

In [46]:
final_dict

{1: [None, 'Incident', 'high', None],
 2: [None, 'Incident', 'medium', None],
 3: [None, 'Incident', 'medium', None],
 4: [None, 'Request', 'medium', None]}

In [50]:
import csv
# Define the header
header = ['ticket_ID', 'department', 'type', 'priority', 'language']

# Write data to CSV file
with open('submission_priority.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for ticket_id, values in final_dict.items():
        writer.writerow([ticket_id] + values)

print("Data has been written to submission.csv")

Data has been written to submission.csv
